# Long-Short Strategy, Part 5: Generating out-of-sample predictions

Implementing, and evaluating a trading strategy for US equities driven by daily return forecasts produced by gradient boosting models.

We'll keep the trading strategy simple and only use a single ML signal.
Improvements would be:
adding multiple signals from different sources, such as complementary ML models trained on different datasets or with different lookahead or lookback periods
Use more sophisticated risk management, from simple stop-loss to value-at-risk analysis


## Imports & Settings

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%matplotlib inline

from time import time
import sys, os
from pathlib import Path

import pandas as pd
from scipy.stats import spearmanr

import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [4]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import MultipleTimeSeriesCV

In [5]:
sns.set_style('whitegrid')

In [6]:
YEAR = 252
idx = pd.IndexSlice

In [7]:
scope_params = ['lookahead', 'train_length', 'test_length']
daily_ic_metrics = ['daily_ic_mean', 'daily_ic_mean_n', 'daily_ic_median', 'daily_ic_median_n']
lgb_train_params = ['learning_rate', 'num_leaves', 'feature_fraction', 'min_data_in_leaf']

## Generate LightGBM predictions

### Model Configuration

In [8]:
base_params = dict(boosting='gbdt',
                   objective='regression',
                   verbose=-1)

categoricals = ['year', 'month', 'sector', 'weekday']

In [9]:
store = Path('data/predictions.h5')

### Get Data

In [10]:
data = pd.read_hdf('data.h5', 'model_data').sort_index()

In [11]:
# Change this value to generate the predictions for different lookahead values (ie. 5, 21 in this case). IMPORTANT to rerun whole file
lookahead = 21

In [12]:
labels = sorted(data.filter(like='_fwd').columns)
features = data.columns.difference(labels).tolist()
label = f'r{lookahead:02}_fwd'

In [13]:
data = data.loc[idx[:, '2010':], features + [label]].dropna()

In [14]:
for feature in categoricals:
    data[feature] = pd.factorize(data[feature], sort=True)[0]

In [15]:
lgb_data = lgb.Dataset(data=data[features],
                       label=data[label],
                       categorical_feature=categoricals,
                       free_raw_data=False)

### Generate predictions

In [16]:
lgb_ic = pd.read_hdf('data/model_tuning.h5', 'lgb/ic')
lgb_daily_ic = pd.read_hdf('data/model_tuning.h5', 'lgb/daily_ic')

In [17]:
def get_lgb_params(data, t=5, best=0):
    param_cols = scope_params[1:] + lgb_train_params + ['boost_rounds']
    df = data[data.lookahead==t].sort_values('ic', ascending=False).iloc[best]
    return df.loc[param_cols]

In [18]:
# Make directory for storing the models for the current lookahead
results_path = Path('TrainedModels', f'{lookahead:02}')
if not results_path.exists():
    results_path.mkdir(parents=True)

In [19]:
# Could change this into a function but the code gets convoluded and saving the files can cause problems, better to rerun file for the different lookaheads 
for position in range(10):
    params = get_lgb_params(lgb_daily_ic,
                            t=lookahead,
                            best=position)

    params = params.to_dict()
    
    for p in ['min_data_in_leaf', 'num_leaves']:
        params[p] = int(params[p])
    train_length = int(params.pop('train_length'))
    test_length = int(params.pop('test_length'))
    num_boost_round = int(params.pop('boost_rounds'))
    params.update(base_params)

    print(f'\nPosition: {position:02}')

    # 1-year out-of-sample period
    n_splits = int(2*YEAR / test_length)
    cv = MultipleTimeSeriesCV(n_splits=n_splits,
                              test_period_length=test_length,
                              lookahead=lookahead,
                              train_period_length=train_length)

    predictions = []
    start = time()
    for i, (train_idx, test_idx) in enumerate(cv.split(X=data), 1):
        print(i, end=' ', flush=True)
        lgb_train = lgb_data.subset(used_indices=train_idx.tolist(),
                                    params=params).construct()

        model = lgb.train(params=params,
                          train_set=lgb_train, 
                          num_boost_round=num_boost_round,
                          verbose_eval=False)
        
        # Output the model data to a txt. file for later signal processing. Takes form of model_#best_#cvfold. Splits the testing period into n_splits, so there will be the same number
        # of models as there are splits. (in this case 8) So model_00_01 will be used to generate predictions for the first period using the first model. 
        pickle_out = open(f"TrainedModels/{lookahead:02}/model_{position:02}_{i:02}.pkl","wb")
        pickle.dump(model, pickle_out)
        pickle_out.close()

        test_set = data.iloc[test_idx, :]
        y_test = test_set.loc[:, label].to_frame('y_test')
        y_pred = model.predict(test_set.loc[:, model.feature_name()])
        predictions.append(y_test.assign(prediction=y_pred))
        

    if position == 0:
        test_predictions = (pd.concat(predictions)
                            .rename(columns={'prediction': position}))
    else:
        test_predictions[position] = pd.concat(predictions).prediction

by_day = test_predictions.groupby(level='date')
for position in range(10):
    if position == 0:
        ic_by_day = by_day.apply(lambda x: spearmanr(
            x.y_test, x[position])[0]).to_frame()
    else:
        ic_by_day[position] = by_day.apply(
            lambda x: spearmanr(x.y_test, x[position])[0])
print(ic_by_day.describe())
test_predictions.to_hdf(store, f'lgb/test/{lookahead:02}')


Position: 00
1 2 3 4 5 6 7 8 
Position: 01
1 2 3 4 5 6 7 8 
Position: 02
1 2 3 4 5 6 7 8 
Position: 03
1 2 3 4 5 6 7 8 
Position: 04
1 2 3 4 5 6 7 8 
Position: 05
1 2 3 4 5 6 7 8 
Position: 06
1 2 3 4 5 6 7 8 
Position: 07
1 2 3 4 5 6 7 8 
Position: 08
1 2 3 4 5 6 7 8 
Position: 09
1 2 3 4 5 6 7 8                 0           1           2           3           4           5  \
count  504.000000  504.000000  504.000000  504.000000  504.000000  504.000000   
mean     0.031678    0.032195    0.032157    0.034329    0.032480    0.057623   
std      0.147520    0.148630    0.125622    0.124657    0.122675    0.127246   
min     -0.367059   -0.365084   -0.337409   -0.327716   -0.329670   -0.286863   
25%     -0.064611   -0.066872   -0.050980   -0.044080   -0.043346   -0.026399   
50%      0.020733    0.023922    0.027135    0.029560    0.029297    0.050958   
75%      0.127254    0.130103    0.113881    0.110636    0.109693    0.137756   
max      0.485846    0.482333    0.485515    0.47762

In [20]:
#The last set of predictions you have run, where the columns represent the values that the top ten models predicted for the lookahead.
test_predictions.head()

y_test         0         1         2         3         4  \
symbol date                                                                     
AAON   2021-08-27 -0.020664 -0.000900  0.000037 -0.013354 -0.015837 -0.016726   
       2021-08-30 -0.043786  0.001564  0.001544 -0.008757 -0.009144 -0.008876   
       2021-08-31 -0.026868  0.003732  0.003525 -0.007347 -0.006118 -0.006775   
       2021-09-01 -0.031857  0.001893  0.002428 -0.004723 -0.010599 -0.014170   
       2021-09-02 -0.022741  0.001845  0.002399 -0.007806 -0.019833 -0.023078   

                          5         6         7         8         9  
symbol date                                                          
AAON   2021-08-27 -0.004766 -0.016734  0.000317 -0.004895  0.001015  
       2021-08-30  0.000681 -0.008955  0.000779 -0.002756  0.002170  
       2021-08-31  0.001347 -0.006633  0.003040  0.000537  0.003915  
       2021-09-01 -0.028460 -0.014252  0.014582  0.003575  0.003438  
       2021-09-02 -0.032663 -0.022672  0.013756 -0.004315  0.002751

Example for isolating the prediction signals from the model:

In [21]:
# Example for creating readable prediction signals, .mean() to average the signals from the top 10 models. Could possibly use weighted average here.
predictions = test_predictions.drop('y_test', axis=1)
predictions = (predictions.loc[~predictions.index.duplicated()]
                   .iloc[:, :10]
                   .mean(1)
                   .sort_index()
                   .dropna()
                  .to_frame('prediction'))
predictions.head()

prediction
symbol date                  
AAON   2019-11-26    0.036209
       2019-11-27    0.035766
       2019-11-29    0.042034
       2019-12-02   -0.005195
       2019-12-03   -0.003896